In [ ]:
TMVA::Tools::Instance();

In [ ]:
TString methodName = "BDTGhiggs120"
//TString methodName = "BDTGhiggs"

In [ ]:
TFile * inputFileS = TFile::Open("sig_1000.root");
TFile * inputFileB = TFile::Open("bkg_800.root");
//TFile * inputFileD = TFile::Open("ntuple_muData.root");
//TFile * inputFileST = TFile::Open("all_ST.root");

In [ ]:
auto sig_tree = (TTree*) inputFileS->Get("AnalysisTree");
auto bkg_tree = (TTree*) inputFileB->Get("AnalysisTree");
//auto ST_tree = (TTree*) inputFileST->Get("base/KinFit");

//auto data_tree = (TTree*) inputFileD->Get("base/KinFit");


In [ ]:
TMVA::Reader reader( "!Color:!Silent" );

In [ ]:
Float_t         Weight_lumi;
Float_t         Reco_ht;
Float_t         Reco_st;
Float_t         Photon_et;
Float_t         Reco_mass_T;

In [ ]:
reader.AddVariable("Reco_ht",&Reco_ht);
reader.AddVariable("Reco_st",&Reco_st);
reader.AddVariable("Photon_et",&Photon_et);
reader.AddVariable("Reco_mass_T",&Reco_mass_T);

In [ ]:
TString weightfile = "dataset/weights/TMVA_TT_Classification_" + methodName + ".weights.xml"

In [ ]:
reader.BookMVA( methodName,weightfile);

In [ ]:
auto h1 = new TH1D("ttbar","Classifier output on Background",40,-1,1);
auto h2 = new TH1D("higgs120","Classifier output on Signal",40,-1,1);
//auto hd = new TH1D("data","Classifier output on Data",40,-1,1);
//auto hST = new TH1D("ST","Classifier output on ST",40,-1,1);
std::vector<float> vout;
std::vector<float> vout2;
std::vector<float> vout3;
std::vector<float> vout4;

In [ ]:
auto readEvents = [&](TTree & tree, TH1 & histo) {
    tree.SetBranchAddress("Weight_lumi",&Weight_lumi);
    tree.SetBranchAddress("Reco_ht",&Reco_ht);
    tree.SetBranchAddress("Reco_st",&Reco_st);
    tree.SetBranchAddress("Photon_et",&Photon_et);
    tree.SetBranchAddress("Reco_mass_T",&Reco_mass_T);
    vout.clear(); 
    vout.resize(tree.GetEntries());
    for (Long64_t ievt=0; ievt < tree.GetEntries();ievt++) {
      if (ievt%500 == 0) std::cout << "--- ... Processing event: " << ievt << std::endl;
      tree.GetEntry(ievt);
      auto output = reader.EvaluateMVA(methodName);
      //if(output<0.4)continue;
        histo.Fill(output,Weight_lumi);
       //histo.Fill(m_j1j2,Weight);
        vout[ievt] = output;  
    }
};

In [ ]:
// read events and fill histogram
readEvents(*bkg_tree,*h1);

In [ ]:
std::vector<float> bkg_result = vout;
//h1->Scale(1/h1->Integral());

In [ ]:
//h1->Scale(1/0.1215);
h1->Draw();
h1->SetLineStyle(1);
gPad->Draw();


gStyle->SetOptStat(1111111);

In [ ]:
readEvents(*sig_tree,*h2);

In [ ]:
h2->SetLineColor(kRed);
h2->Scale(1/0.1215);
std::vector<float> sig_result = vout;


In [ ]:
//readEvents(*data_tree,*hd);

In [ ]:
//hd->SetLineColor(1);
//hd->SetLineStyle(2);
//h2->Scale(1/0.1215);
//std::vector<float> data_result = vout;

In [ ]:
//readEvents(*ST_tree,*hST);

In [ ]:
 //h2->Draw();
h2->Draw("SAME");
//hST->Draw("SAME");
//hd->Draw("SAME");

gPad->Draw();
gPad->BuildLegend();


TH1F *hdata = (TH1F*)h1->Clone("data_obs");



In [ ]:
//void writehistlist()

   //TList *l = new TList();
   //TH1F *h1 = new TH1F("h1","h1",100.,0.,1.);
   //TH1F *h2 = new TH1F("h2","h2",100.,0.,1.);
   //TH1F *h3 = new TH1F("h3","h3",100.,0.,1.);
   //l->Add(h1);
   //l->Add(h2);
   //l->Add(h3);
   TFile *f = new TFile("histlistBDT.root","RECREATE");
   h1->Write();
   h2->Write() ;  
   //hd->Write();
   f->ls();



In [ ]:
f->Close();

In [ ]:
//b->Close();

In [ ]:
//TFile f1("ntuple_bkg_ttbar.root","new");

In [ ]:
//f1.Write("h1")

In [ ]:
//f1.Close()

In [ ]:
//TFile f("histbkg.root","RECREATE");

In [ ]:
//h1->Write()

In [ ]:
//f.Close()

In [ ]:
//TFile g("histsig.root","RECREATE");

In [ ]:
//h2->Write()

In [ ]:
//g.Close()

In [ ]:
// TFile f("ntuple_bkg_ttbar.root","new");

In [ ]:
TMVA::ROCCurve roc(sig_result, bkg_result);
TGraph *  roc_graph  = roc.GetROCCurve(); 
roc_graph->Draw("AL");

In [ ]:
gPad->Draw();
std::cout << "ROC integral = " << roc.GetROCIntegral() << std::endl;